# Creating Data for the TuningTools package

The TuningTools package uses numpy as the primary data source for tuning the particle selection algorithms. This documentation concerns the data transformation from a ROOT file (PhysVal) or, in a future release, directly from xAOD to the used tuning data format.

Basically, there are two ways of creating the tuning data file. You can either:
- [[Recommended] Run the shell createData.py executable](#Using-the-createData.py-executable); or
- [Interact with the python code](#Interacting-with-the-python-code).

<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

# Retrieve data from the GRID

It is most likely that you have generated the data on the GRID. The approach for running the data transformation on the GRID is currently under-development, so you will have to download and run the transformation on standalone, then upload it again to the GRID. Either way, if you want to run the tuning on standalone, this is be needed, so in order to optimize the process while implementation is not finished, you can follow the next steps.

## Optimal GRID PhysVal download

The standalone approaches consider that you have already downloaded the data from the reconstructions. 

If you need to download the PhysVals on the GRID, you can make use of the `scripts/grid_scripts/run_dump.py` script. This simple script can be used to download the PhysVals in batches, and skimming them (thus reducing their size) to keep only the needed TTrees.

An usage example of a dataset download while also removing all Trigger information can be done as follows:

```bash
run_dump.py --inDS user.wsfreund.mc14_13TeV.129160.Pythia8_AU2CTEQ6L1_perf_JF17.e3084_s2044_s2008_r5988.rr0003.ph0002_PhysVal/ --outFolder mergedJet --triggerList '' --numberOfSamplesPerPackage 250
```
It is also possible to specify the trigger list to filter using `--trigerList='e24_lhmedium_L1EM18VH' 'e24_lhtight_L1EM18VH'`...

The full command options are the following:


In [1]:
%%bash
run_dump.py -- -h

usage: run_dump.py [-h] --inDS INDS [--outFolder OUTFOLDER] [--triggerList TRIGGERLIST [TRIGGERLIST ...]] [--numberOfSamplesPerPackage NUMBEROFSAMPLESPERPACKAGE]
run_dump.py: error: argument --inDS is required


# The createData.py executable

This is the preferred and supported way for creating the data files. You will need to have the xAOD/PhysVal with the rings in order to generate the data files. The command can be run after you set the environment through `source setrootcore.sh` command on the RootCore packages directory.

The help information should be self-explanatory, where the most important options are `--sgnInputFiles`, `--bkgInputFiles`, `--operation`, `--reference` and `--treePath`.

If you want to run a $\eta$ and $E_T$ dependent discrimination, you can specify the bins through `--etaBins` and `etBins`, respectively.


In [2]:
%%bash
createData.py

usage: createData.py -s SignalInputFiles [SignalInputFiles ...] -b BackgroundInputFiles [BackgroundInputFiles ...] -op OPERATION [-t TreePath [TreePath ...]]
                     [--reference {Off_CutID,Off_Likelihood,Truth,_ignoreCase} [{Off_CutID,Off_Likelihood,Truth,_ignoreCase} ...]] [-tEff EfficienciyTreePath [EfficienciyTreePath ...]] [-l1 L1EMCLUSCUT]
                     [-l2 L2ETCUT] [-off OFFETCUT] [--getRatesOnly] [--etBins ETBINS [ETBINS ...]] [--etaBins ETABINS [ETABINS ...]] [--ringConfig RINGCONFIG [RINGCONFIG ...]] [-nC NCLUSTERS]
                     [-o OUTPUT] [--crossFile CROSSFILE] [--output-level {DEBUG,ERROR,FATAL,INFO,VERBOSE,WARNING,_ignoreCase}]

Create TuningTool data from PhysVal.

optional arguments:
  --output-level {DEBUG,ERROR,FATAL,INFO,VERBOSE,WARNING,_ignoreCase}
                        The output level for the main logger

Required arguments:

  -s SignalInputFiles [SignalInputFiles ...], --sgnInputFiles SignalInputFiles [SignalInputFiles ...]
      

# Interacting with the python code


If, instead, you want to directly call the python code within a script. This is done through the `__call__` method of the `CreateData` class:

In [3]:
from TuningTools.CreateData import createData
help(createData.__call__)

Help on method __call__ in module TuningTools.CreateData:

__call__(self, sgnFileList, bkgFileList, ringerOperation, **kw) method of TuningTools.CreateData.CreateData instance
    Creates a numpy file ntuple with rings and its targets
    Arguments:
      - sgnFileList: A python list or a comma separated list of the root files
          containing the TuningTool TTree for the signal dataset
      - bkgFileList: A python list or a comma separated list of the root files
          containing the TuningTool TTree for the background dataset
      - ringerOperation: Set Operation type to be used by the filter
    Optional arguments:
      - output ['tuningData']: Name for the output file
      - referenceSgn [Reference.Truth]: Filter reference for signal dataset
      - referenceBkg [Reference.Truth]: Filter reference for background dataset
      - treePath [Set using operation]: set tree name on file, this may be set to
        use different sources then the default.
          Default for:


## Using ReadData

However, in some very special cases, you might want to run directly the `ReadData` class instead, to obtain only some of the datasets rings.

Its documentation should cover all configuration:

In [4]:
from TuningTools.ReadData import readData
help(readData.__call__)

Help on method __call__ in module TuningTools.ReadData:

__call__(self, fList, ringerOperation, **kw) method of TuningTools.ReadData.ReadData instance
    Returns ntuple with rings and its targets
    Arguments:
      - fList: The file path or file list path. It can be an argument list of 
      two types:
        o List: each element is a string path to the file;
        o Comma separated string: each path is separated via a comma
        o Folders: Expand folders recursively adding also files within them to analysis
      - ringerOperation: Set Operation type. It can be both a string or the
        RingerOperation
    Optional arguments:
      - filterType [None]: whether to filter. Use FilterType enumeration
      - reference [Truth]: set reference for targets. Use Reference enumeration
      - treePath [Set using operation]: set tree name on file, this may be set to
        use different sources then the default.
          Default for:
            o Offline: Offline/Egamma/Ntuple/e

# The information available on the TuningDataArchieve

We can explore the information available on the TuningDataArchieve by using the RingerCore.FileIO load method directly on the file and checking its RAW content:

In [9]:
from RingerCore.FileIO import load
f = load('tuningData.npz')
print f
f.keys()

['signal_patterns_etBin_0_etaBin_2',
 'signal_patterns_etBin_0_etaBin_3',
 'signal_patterns_etBin_0_etaBin_0',
 'signal_patterns_etBin_0_etaBin_1',
 'background_patterns_etBin_2_etaBin_0',
 'background_patterns_etBin_2_etaBin_1',
 'background_patterns_etBin_2_etaBin_2',
 'background_patterns_etBin_2_etaBin_3',
 'signal_patterns_etBin_2_etaBin_0',
 'signal_patterns_etBin_2_etaBin_1',
 'signal_patterns_etBin_2_etaBin_2',
 'signal_patterns_etBin_2_etaBin_3',
 'background_efficiencies',
 'version',
 'signal_patterns_etBin_1_etaBin_1',
 'signal_patterns_etBin_1_etaBin_0',
 'signal_patterns_etBin_1_etaBin_3',
 'signal_patterns_etBin_1_etaBin_2',
 'type',
 'et_bins',
 'eta_bins',
 'background_cross_efficiencies',
 'background_patterns_etBin_0_etaBin_2',
 'background_patterns_etBin_0_etaBin_3',
 'background_patterns_etBin_0_etaBin_0',
 'background_patterns_etBin_0_etaBin_1',
 'background_patterns_etBin_1_etaBin_1',
 'background_patterns_etBin_1_etaBin_0',
 'background_patterns_etBin_1_etaBin_3

In this version, we are saving each one of the `signal` and `background` rings data in a dedicated bin file, so that the rings information can be retrieved without loading the full data into memory. The other information available on the file, which are quite small in the memory, are not separated into bins. You will find, besides the rings, the `et_bins` and `eta_bins` limits, the `background` and `signal` benchmark efficiencies, which, for the trigger, will be `L2CaloAccept`, `L2ElAccept`, `EFCaloAccept` and `EFElAccept`; and, for the offline, they will be `CutIDLoose`, `CutIDMedium`, `CutIDTight`, `LHLoose`, `LHMedium` and `LHTight`. Those efficiencies are measured for all the dataset and given with respect to the used reference. However, if you provide the `CrossValid` object that will be used to tune the Ringer selector, there will be available the `background` and `signal` cross-validation datasets efficiencies to compare efficiencies in the same used datasets. 

One more important note: when loading only one bin into the memory, the context manager will handle and keep only the desired bin information into memory, as we can see in the following code:

In [10]:
from TuningTools.CreateData import TuningDataArchieve
with TuningDataArchieve("tuningData.npz", eta_bin = 0, et_bin = 0) as data: pass
print 'Background L2Calo in(et=0,eta=0) efficiency is: ', data['background_efficiencies']['L2CaloAccept'].eff_str()
print 'Background HLT bin(et=0,eta=0) efficiency is: ', data['background_efficiencies']['EFElAccept'].eff_str()
print 'Signal L2Calo bin(et=0,eta=0) efficiency is: ', data['signal_efficiencies']['L2CaloAccept'].eff_str()
print 'Signal HLT bin(et=0,eta=0) efficiency is: ', data['signal_efficiencies']['EFElAccept'].eff_str()
data

Background L2Calo in(et=0,eta=0) efficiency is:  1.398601 (6/429)
Background HLT bin(et=0,eta=0) efficiency is:  0.233100 (1/429)
Signal L2Calo bin(et=0,eta=0) efficiency is:  72.950820 (89/122)
Signal HLT bin(et=0,eta=0) efficiency is:  60.655738 (74/122)


{'background_cross_efficiencies': OrderedDict([('L2CaloAccept',
               <TuningTools.ReadData.BranchCrossEffCollector at 0x27af390>),
              ('L2ElAccept',
               <TuningTools.ReadData.BranchCrossEffCollector at 0x5b6df90>),
              ('EFCaloAccept',
               <TuningTools.ReadData.BranchCrossEffCollector at 0x647c350>),
              ('EFElAccept',
               <TuningTools.ReadData.BranchCrossEffCollector at 0x5b73d10>)]),
 'background_efficiencies': OrderedDict([('L2CaloAccept',
               <TuningTools.ReadData.BranchEffCollector at 0x53c5710>),
              ('L2ElAccept',
               <TuningTools.ReadData.BranchEffCollector at 0x53c5650>),
              ('EFCaloAccept',
               <TuningTools.ReadData.BranchEffCollector at 0x53c5390>),
              ('EFElAccept',
               <TuningTools.ReadData.BranchEffCollector at 0x53c5750>)]),
 'background_patterns': array([[  192.13546753,  1005.24487305,     2.97116709, ...,
           -42.

We can also do not request to load a specific bin, but rather the full data available for all bins in the file:

In [11]:
from TuningTools.CreateData import TuningDataArchieve
with TuningDataArchieve("tuningData.npz") as data: pass
print 'Background L2Calo in(et=1,eta=0) efficiency is: ', data['background_efficiencies']['L2CaloAccept'][1][0].eff_str()
print 'Background HLT bin(et=1,eta=0) efficiency is: ', data['background_efficiencies']['EFElAccept'][1][0].eff_str()
print 'Signal L2Calo bin(et=1,eta=0) efficiency is: ', data['signal_efficiencies']['L2CaloAccept'][1][0].eff_str()
print 'Signal HLT bin(et=1,eta=0) efficiency is: ', data['signal_efficiencies']['EFElAccept'][1][0].eff_str()
data

Background L2Calo in(et=1,eta=0) efficiency is:  1.265823 (2/158)
Background HLT bin(et=1,eta=0) efficiency is:  0.000000 (0/158)
Signal L2Calo bin(et=1,eta=0) efficiency is:  99.180328 (484/488)
Signal HLT bin(et=1,eta=0) efficiency is:  96.926230 (473/488)


{'background_cross_efficiencies': OrderedDict([('L2CaloAccept',
               [[<TuningTools.ReadData.BranchCrossEffCollector at 0x5dee1d0>,
                 <TuningTools.ReadData.BranchCrossEffCollector at 0x6623e90>]]),
              ('L2ElAccept',
               [[<TuningTools.ReadData.BranchCrossEffCollector at 0x6625890>,
                 <TuningTools.ReadData.BranchCrossEffCollector at 0x6098590>]]),
              ('EFCaloAccept',
               [[<TuningTools.ReadData.BranchCrossEffCollector at 0x6099f10>,
                 <TuningTools.ReadData.BranchCrossEffCollector at 0x60abc10>]]),
              ('EFElAccept',
               [[<TuningTools.ReadData.BranchCrossEffCollector at 0x60ad650>,
                 <TuningTools.ReadData.BranchCrossEffCollector at 0x60bf310>]])]),
 'background_efficiencies': OrderedDict([('L2CaloAccept',
               [[<TuningTools.ReadData.BranchEffCollector at 0x63e6350>,
                 <TuningTools.ReadData.BranchEffCollector at 0x63e65d0>]]),
  

Please note that loading all information is not needed, generally, as the bin information is treated separately for each tuning.

# Examples

## Example 1: Creating data using the command line

The following example retrieves data from two folders filled with files of two full runs of simulated data specified as signal (`-s`) and background (`-b`). 

The algorithm is specified to operate in the fast calorimeter step (`-op L2`) and the references used to filter the targets are the offline likelihood and the truth for, respectively, the signal and background datasets. 

It is specified a cut of 19 GeV on L2 cluster measured energy, so the algorithm will be trained only with candidates that pass this threshold. This cut should be the threshold applied on the chain to avoid passing candidates with lower energy that the selector will receive during its operation. A energy cut of 20 GeV is required on L1EMCluster, to operate closer of a L1EM20VH.

The trigger chains are specified for the signal and background datasets afterwards through the `-t` flag.

A cross validation file is passed for also measuring the baseline efficiency on the datasets that will be used for training. If this is not specified, the baseline efficiencies will be measured only for the full dataset.

A maximum number of clusters is specified (1000), as this is just an example. 

The $E_T$ and $|\eta|$ bins are specified through the `--etBins` and `--etaBins` flags.

In [3]:
%%bash
createData.py \
  -s /afs/cern.ch/work/j/jodafons/public/Online/PhysVal/user.jodafons.mc14_13TeV.147406.PowhegPythia8_AZNLO_Zee.recon.RDO.rel20.1.0.4.e3059_s1982_s2008_r5993_rr0104_a0001_PhysVal.root  \
  -b /afs/cern.ch/work/j/jodafons/public/Online/PhysVal/user.jodafons.mc14_13TeV.129160.Pythia8_AU2CTEQ6L1_perf_JF17.recon.RDO.rel20.1.0.4.e3084_s2044_s2008_r5988.rr0104_a0001_PhysVal.root \
  -op L2 \
  --reference Off_Likelihood Truth \
  -l1 20 \
  -l2 19 \
  -t Trigger/HLT/Egamma/ZeeNtuple/e24_lhmedium_ringer_L1EM18VH \
     Trigger/HLT/Egamma/BackgroundNtuple/e24_lhmedium_ringer_L1EM18VH \
  --crossFile /afs/cern.ch/work/w/wsfreund/private/crossValid.pic.gz \
  --nClusters 10000 \
  --etBins 0 30  50  20000 \
  --etaBins 0  0.8   1.37  1.54  2.5
  #--output-level VERBOSE

Py.CreateData                           INFO Extracting signal dataset information...
Py.ReadData                         INFO There is available a total of 306345 entries.
Py.ReadData                         INFO L2CaloAccept_etBin0_etaBin0 : 78.623719 (537/683)
Py.ReadData                         INFO L2ElAccept_etBin0_etaBin0 : 0.000000 (0/683)
Py.ReadData                         INFO EFCaloAccept_etBin0_etaBin0 : 78.623719 (537/683)
Py.ReadData                         INFO EFElAccept_etBin0_etaBin0 : 60.907760 (416/683)
Py.ReadData                         INFO L2CaloAccept_etBin0_etaBin1 : 80.940594 (327/404)
Py.ReadData                         INFO L2ElAccept_etBin0_etaBin1 : 0.000000 (0/404)
Py.ReadData                         INFO EFCaloAccept_etBin0_etaBin1 : 80.940594 (327/404)
Py.ReadData                         INFO EFElAccept_etBin0_etaBin1 : 66.336634 (268/404)
Py.ReadData                         INFO L2CaloAccept_etBin0_etaBin2 : 80.000000 (60/75)
Py.ReadData             

Note that a file mc14_13TeV.147406.129160.sgn.offLikelihood.bkg.truth.trig.e24_lhmedium_nod0_L1EM20VH.npz was created, containing the tuning data to be used by the `TuningJob.py`. This file also contains the efficiencies benchmarks, as we will check on the information available on the [`TuningDataArchieve`](#The-information-available-on-the-TuningDataArchieve).

In [17]:
%%bash
ls

CreateData.ipynb
mc14_13TeV.147406.129160.sgn.offLikelihood.bkg.truth.trig.e24_lhmedium_nod0_L1EM18VH.npz
Tuning.ipynb


## Example 2: Creating data using a python script

The same example as above, but using python script:

In [19]:
# %load ../scripts/skeletons/create_data.py
#!/usr/bin/env python
from TuningTools.ReadData import *
from TuningTools.CreateData import createData
from RingerCore.Logger import LoggingLevel

from TuningTools.CrossValid import CrossValidArchieve
with CrossValidArchieve( "/afs/cern.ch/work/w/wsfreund/private/crossValid.pic.gz" ) as CVArchieve:
  crossVal = CVArchieve
del CVArchieve

RatesOnly=False
etaBins  = [0, 0.8 , 1.37, 1.54, 2.5]
etBins   = [0, 30, 50, 20000] # in GeV
output   = 'mc14_13TeV.147406.129160.sgn.offLikelihood.bkg.truth.trig.e24_lhmedium_nod0_L1EM18VH'
basepath = '/afs/cern.ch/work/j/jodafons/public/Online/PhysVal/'
bkgName  = 'user.jodafons.mc14_13TeV.129160.Pythia8_AU2CTEQ6L1_perf_JF17.recon.RDO.rel20.1.0.4.e3084_s2044_s2008_r5988.rr0104_a0001_PhysVal.root'
sgnName  = 'user.jodafons.mc14_13TeV.147406.PowhegPythia8_AZNLO_Zee.recon.RDO.rel20.1.0.4.e3059_s1982_s2008_r5993_rr0104_a0001_PhysVal.root'
treePathList = ['Trigger/HLT/Egamma/ZeeNtuple/e24_lhmedium_ringer_L1EM18VH', \
                'Trigger/HLT/Egamma/BackgroundNtuple/e24_lhmedium_ringer_L1EM18VH']

createData( basepath + '/' + sgnName, 
            basepath + '/' + bkgName,
            RingerOperation.L2,
            referenceSgn       = Reference.Off_Likelihood,
            referenceBkg       = Reference.Truth,
            treePath           = treePathList,
            l1EmClusCut        = 0,
            l2EtCut            = 19,
            # level              = LoggingLevel.DEBUG,
            nClusters          = 2000,
            getRatesOnly       = RatesOnly,
            etBins             = etBins,
            etaBins            = etaBins,
            crossVal           = crossVal )




Py.CreateData                           INFO Extracting signal dataset information...
Py.ReadData                         INFO There is available a total of 306345 entries.
Py.ReadData                         INFO L2CaloAccept_etBin0_etaBin0 : 72.950820 (89/122)
Py.ReadData                         INFO L2ElAccept_etBin0_etaBin0 : 0.000000 (0/122)
Py.ReadData                         INFO EFCaloAccept_etBin0_etaBin0 : 72.950820 (89/122)
Py.ReadData                         INFO EFElAccept_etBin0_etaBin0 : 60.655738 (74/122)
Py.ReadData                         INFO L2CaloAccept_etBin0_etaBin1 : 81.250000 (65/80)
Py.ReadData                         INFO L2ElAccept_etBin0_etaBin1 : 0.000000 (0/80)
Py.ReadData                         INFO EFCaloAccept_etBin0_etaBin1 : 81.250000 (65/80)
Py.ReadData                         INFO EFElAccept_etBin0_etaBin1 : 60.000000 (48/80)
Py.ReadData                         INFO L2CaloAccept_etBin0_etaBin2 : 86.363636 (19/22)
Py.ReadData                       

This time we've created the default output file (`tuningData.npz`), as can be seen with `ls` command:


In [20]:
%%bash
ls

CreateData.ipynb
mc14_13TeV.147406.129160.sgn.offLikelihood.bkg.truth.trig.e24_lhmedium_nod0_L1EM18VH.npz
tuningData.npz
Tuning.ipynb


## Example 3: Direct use of ReadData 

Here, we run a similar example as above, but using the ReadData.

In [7]:
# %load ../scripts/analysis_scripts/Trigger_20_0_1_4/ReadData.py
#!/usr/bin/env python
from RingerCore.FileIO import save, load
from TuningTools.ReadData import *
import numpy as np

etaBins = [0, 0.8 , 1.37, 1.54, 2.5]
etBins  = [0,30, 50, 20000]# in GeV

#etaBins=[0.0,2.50]
#etBins=[0,200000]

output   = 'mc14_13TeV.147406.129160.sgn.offLikelihood.bkg.truth.trig.e24_lhmedium_L1EM20VH'
basepath = '/afs/cern.ch/work/j/jodafons/public/Online/PhysVal'
#bkgName='user.jodafons.mc14_13TeV.129160.Pythia8_AU2CTEQ6L1_perf_JF17.recon.RDO.rel20.1.0.4.e3084_s2044_s2008_r5988.rr0040_ph0002_PhysVal.root'
#sgnName='user.jodafons.mc14_13TeV.147406.PowhegPythia8_AZNLO_Zee.recon.RDO.rel20.1.0.4.e3059_s1982_s2008_r5993_rr0040_ph0002_PhysVal.root'
bkgName='user.jodafons.mc14_13TeV.129160.Pythia8_AU2CTEQ6L1_perf_JF17.recon.RDO.rel20.1.0.4.e3084_s2044_s2008_r5988.rr0104_a0001_PhysVal.root'
sgnName='user.jodafons.mc14_13TeV.147406.PowhegPythia8_AZNLO_Zee.recon.RDO.rel20.1.0.4.e3059_s1982_s2008_r5993_rr0104_a0001_PhysVal.root'

print 'Background:'

npBkg, bkgSummary, _  = readData(basepath+'/'+bkgName, 
                         RingerOperation.L2,
                         treePath= 'Trigger/HLT/Egamma/BackgroundNtuple/e24_lhmedium_ringer_perf_L1EM20VH', 
                         #l1EmClusCut = 20, 
                         l2EtCut = 19,
                         filterType = FilterType.Background, 
                         reference = Reference.Truth,
                         etaBins=etaBins,
                         etBins=etBins,
                         nClusters=200,
                         #getRatesOnly=True,
                         )


print 'Signal:'

npSgn, sgnSummary, _  = readData(basepath+'/'+sgnName,
                         RingerOperation.L2,
                         treePath = 'Trigger/HLT/Egamma/ZeeNtuple/e24_lhmedium_ringer_perf_L1EM20VH',
                         #l1EmClusCut = 20,
                         l2EtCut = 19,
                         filterType = FilterType.Signal,
                         reference = Reference.Off_Likelihood,
                         etaBins=etaBins,
                         etBins=etBins,
                         nClusters=200,
                         #getRatesOnly=True,
                         )

summary = {'sgn':sgnSummary,'bkg':bkgSummary}
save(summary, output+'_summary')


from TuningTools.CreateData import TuningDataArchieve
import scipy.io as sio
DoMatlab=True

for cEt in range(len(etBins)-1):
  for cEta in range(len(etaBins)-1):
    sufix=('_etBin_%d_etaBin_%d')%(cEt,cEta)
    print ('Saving position: [%d][%d]')%(cEt,cEta)
    if npSgn[cEt][cEta] is None:
      npSgn[cEt][cEta] = np.array([[]])
    print 'sgn shape is ',npSgn[cEt][cEta].shape
    if npBkg[cEt][cEta] is None:
      npBkg[cEt][cEta] = np.array([[]])
    print 'bkg shape is ',npBkg[cEt][cEta].shape
    savedPath = TuningDataArchieve( output+sufix,
                                    signal_patterns = npSgn[cEt][cEta],
                                    background_patterns = npBkg[cEt][cEta] ).save()

    if DoMatlab:
      obj = {'signal_patterns':npSgn[cEt][cEta],'background_patterns':npBkg[cEt][cEta]}
      sio.savemat(output+sufix+'.mat',obj)

    print ('Saved path is %s')%(savedPath)



Py.ReadData                         INFO There is available a total of 15321 entries.
Py.ReadData                         INFO L2CaloAccept_etBin0_etaBin0 : 100.000000 (48/48)
Py.ReadData                         INFO L2ElAccept_etBin0_etaBin0 : 0.000000 (0/48)
Py.ReadData                         INFO EFCaloAccept_etBin0_etaBin0 : 100.000000 (48/48)
Py.ReadData                         INFO EFElAccept_etBin0_etaBin0 : 0.000000 (0/48)
Py.ReadData                         INFO L2CaloAccept_etBin0_etaBin1 : 100.000000 (28/28)
Py.ReadData                         INFO L2ElAccept_etBin0_etaBin1 : 0.000000 (0/28)
Py.ReadData                         INFO EFCaloAccept_etBin0_etaBin1 : 100.000000 (28/28)
Py.ReadData                         INFO EFElAccept_etBin0_etaBin1 : 0.000000 (0/28)
Py.ReadData                         INFO L2CaloAccept_etBin0_etaBin2 : 100.000000 (1/1)
Py.ReadData                         INFO L2ElAccept_etBin0_etaBin2 : 0.000000 (0/1)
Py.ReadData                         INFO E

Note that the previous script saved each of the bins in a separated file. It also exported them into matlab format:

In [8]:
%%bash
ls

CreateData.ipynb
mc14_13TeV.147406.129160.sgn.offLikelihood.bkg.truth.trig.e24_lhmedium_L1EM20VH_etBin_0_etaBin_0.mat
mc14_13TeV.147406.129160.sgn.offLikelihood.bkg.truth.trig.e24_lhmedium_L1EM20VH_etBin_0_etaBin_0.npz
mc14_13TeV.147406.129160.sgn.offLikelihood.bkg.truth.trig.e24_lhmedium_L1EM20VH_etBin_0_etaBin_1.mat
mc14_13TeV.147406.129160.sgn.offLikelihood.bkg.truth.trig.e24_lhmedium_L1EM20VH_etBin_0_etaBin_1.npz
mc14_13TeV.147406.129160.sgn.offLikelihood.bkg.truth.trig.e24_lhmedium_L1EM20VH_etBin_0_etaBin_2.mat
mc14_13TeV.147406.129160.sgn.offLikelihood.bkg.truth.trig.e24_lhmedium_L1EM20VH_etBin_0_etaBin_2.npz
mc14_13TeV.147406.129160.sgn.offLikelihood.bkg.truth.trig.e24_lhmedium_L1EM20VH_etBin_0_etaBin_3.mat
mc14_13TeV.147406.129160.sgn.offLikelihood.bkg.truth.trig.e24_lhmedium_L1EM20VH_etBin_0_etaBin_3.npz
mc14_13TeV.147406.129160.sgn.offLikelihood.bkg.truth.trig.e24_lhmedium_L1EM20VH_etBin_1_etaBin_0.mat
mc14_13TeV.147406.129160.sgn.offLikelihood.bkg.truth.trig.e24_lhmedium_L1E

<script type="text/javascript">
    show=true;
    function toggle(){
        if (show){
            $('div.input').hide();
        }else{
            $('div.input').show();
        }
        show = !show
    }
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')
</script>
<a href="javascript:toggle()" target="_self"></a>